# BÁO CÁO: PHÂN TÍCH DỮ LIỆU VÀ XÂY DỰNG PIPELINE (DATA PREPARATION)

**Mục tiêu:**
1.  **Data Understanding:** Tải và làm sạch dữ liệu thô.
2.  **Exploratory Data Analysis (EDA):** Phân tích các phân phối và tương quan dưới dạng cấu trúc dữ liệu (JSON-ready) phục vụ Dashboard.
3.  **Business Analytics:** Tính toán các chỉ số KPI kinh doanh (Revenue, Churn Rate).
4.  **Feature Engineering:** Xây dựng Pipeline chuẩn hóa dữ liệu cho Model.

[Image of data pipeline architecture]

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import set_config

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
set_config(display='diagram') # Hiển thị Pipeline dạng sơ đồ khối
plt.style.use('seaborn-v0_8-whitegrid')

# Đường dẫn dữ liệu (Cập nhật lại cho phù hợp với môi trường Notebook)
DATA_PATH = "../data/Customer-Churn.csv" 
MODELS_DIR = "../models"
os.makedirs(MODELS_DIR, exist_ok=True)

## 1. Tải và Làm sạch Dữ liệu
Dữ liệu được tải từ CSV. Các bước làm sạch bao gồm:
* Xóa khoảng trắng thừa trong tên cột.
* Chuyển đổi các chuỗi rỗng thành `NaN`.
* Ép kiểu dữ liệu cho `TotalCharges` và `SeniorCitizen`.

In [ ]:
def load_data(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Không tìm thấy file tại: {path}")

    df = pd.read_csv(path)
    # Chuẩn hóa tên cột
    df.columns = df.columns.str.strip()
    
    # Xử lý giá trị rỗng ẩn (khoảng trắng)
    df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

    # Ép kiểu dữ liệu số
    if 'TotalCharges' in df.columns:
        df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    if 'SeniorCitizen' in df.columns:
        df['SeniorCitizen'] = pd.to_numeric(df['SeniorCitizen'], errors='coerce')

    return df

# Thực thi
try:
    df = load_data(DATA_PATH)
    print(f"Kích thước dữ liệu: {df.shape}")
    display(df.head(3))
except Exception as e:
    print(f"Lỗi: {e}")

## 2. Phân tích Khám phá (EDA)
Hàm `perform_eda` được thiết kế để trả về dữ liệu dạng Dictionary (cấu trúc JSON), giúp dễ dàng tích hợp vào API.
Dưới đây, chúng ta sẽ gọi hàm này và trực quan hóa kết quả trả về.

In [ ]:
def perform_eda(df):
    """Trả về dữ liệu EDA dạng Dictionary (Dashboard-ready)"""
    
    # 1. Phân phối Target
    target_dist = df['Churn'].value_counts(normalize=True).to_dict() if 'Churn' in df.columns else {}
    
    # 2. Tương quan (Correlation)
    correlations = {}
    if 'Churn' in df.columns:
        churn_flag = df['Churn'].map({'Yes': 1, 'No': 0})
        numeric_df = df.select_dtypes(include=[np.number]).copy()
        numeric_df['Churn_Flag'] = churn_flag
        # Tính correlation
        corr_matrix = numeric_df.corr()
        if 'Churn_Flag' in corr_matrix.columns:
            correlations = corr_matrix['Churn_Flag'].drop('Churn_Flag').sort_values(ascending=False).to_dict()

    # 3. Categorical Analysis (Churn Rate by Contract)
    churn_by_contract = {}
    if 'Contract' in df.columns and 'Churn' in df.columns:
        grouped = df.groupby('Contract')['Churn'].apply(lambda x: (x == 'Yes').mean())
        churn_by_contract = grouped.to_dict()

    return {
        "target_distribution": target_dist,
        "correlations": correlations,
        "churn_by_contract": churn_by_contract
    }

# Gọi hàm
eda_data = perform_eda(df)
print("Dữ liệu EDA trả về (JSON structure):")
print(eda_data)

In [ ]:
# Vẽ biểu đồ từ dữ liệu EDA trả về
plt.figure(figsize=(15, 5))

# 1. Target Distribution
plt.subplot(1, 3, 1)
names = list(eda_data['target_distribution'].keys())
values = list(eda_data['target_distribution'].values())
plt.bar(names, values, color=['green', 'red'])
plt.title("Phân phối Churn (Target)")
plt.ylabel("Tỷ lệ")

# 2. Correlations
plt.subplot(1, 3, 2)
features = list(eda_data['correlations'].keys())
corr_vals = list(eda_data['correlations'].values())
sns.barplot(x=corr_vals, y=features, palette="coolwarm")
plt.title("Tương quan với Churn")
plt.xlabel("Hệ số tương quan")

# 3. Churn by Contract
plt.subplot(1, 3, 3)
contracts = list(eda_data['churn_by_contract'].keys())
rates = [v * 100 for v in eda_data['churn_by_contract'].values()] # Đổi sang %
sns.barplot(x=contracts, y=rates, palette="magma")
plt.title("Tỷ lệ rời bỏ theo Hợp đồng")
plt.ylabel("Tỷ lệ (%)")

plt.tight_layout()
plt.show()

## 3. Chỉ số Kinh doanh (Business Analytics)
Tính toán các KPI quan trọng như Doanh thu và Tỷ lệ rời bỏ.

In [ ]:
def get_business_analytics(df):
    stats = {
        "Total Revenue": df['TotalCharges'].sum(),
        "Avg Revenue/User": df['MonthlyCharges'].mean(),
        "Total Customers": len(df),
        "Churn Rate (%)": (df['Churn'] == 'Yes').mean() * 100
    }
    return stats

kpis = get_business_analytics(df)

# Hiển thị đẹp
print("=== BUSINESS DASHBOARD ===")
for key, value in kpis.items():
    print(f"{key:<20}: {value:,.2f}")

## 4. Xây dựng Pipeline Tiền xử lý
Sử dụng `ColumnTransformer` để xử lý song song dữ liệu số và phân loại.
* **Numeric:** Điền khuyết (Median) -> Chuẩn hóa (StandardScaler).
* **Categorical:** Điền khuyết (Constant) -> Mã hóa (OneHotEncoder).

In [ ]:
FEATURE_COLUMNS = ['tenure', 'MonthlyCharges', 'Contract', 'InternetService', 'PaymentMethod']

def build_preprocessor(df):
    # Tách X, y
    y = df['Churn'].map({'Yes': 1, 'No': 0})
    X = df[FEATURE_COLUMNS]
    
    # Định nghĩa các nhóm cột
    numeric_features = ['tenure', 'MonthlyCharges']
    categorical_features = ['Contract', 'InternetService', 'PaymentMethod']

    # Pipeline con
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # Bộ xử lý tổng hợp
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])
    
    return preprocessor, X, y

# Build
preprocessor, X, y = build_preprocessor(df)

# Hiển thị sơ đồ Pipeline (Interactive Diagram)
preprocessor

In [ ]:
# Fit dữ liệu
print("Đang huấn luyện bộ tiền xử lý...")
X_processed = preprocessor.fit_transform(X)

print(f"Kích thước dữ liệu gốc: {X.shape}")
print(f"Kích thước sau xử lý : {X_processed.shape}")

# Lưu artifact
save_path = os.path.join(MODELS_DIR, 'preprocessor.joblib')
joblib.dump(preprocessor, save_path)
print(f"Đã lưu Preprocessor tại: {save_path}")